In [1]:
import numpy as np 
import pandas as pd 
import warnings

warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('train.csv')
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
test = pd.read_csv('test.csv')
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [4]:
def find_missing_percent(data):
    """
    Returns dataframe containing the total missing values and percentage of total
    missing values of a column.
    """
    miss_df = pd.DataFrame({'ColumnName':[],'TotalMissingVals':[],'PercentMissing':[]})

    mylist = []
    for col in data.columns:
        sum_miss_val = data[col].isnull().sum()
        percent_miss_val = round((sum_miss_val/data.shape[0])*100, 2)
        mylist.append(dict(zip(miss_df.columns, [col, sum_miss_val, percent_miss_val])))

    df = pd.DataFrame(mylist)

    return df

In [5]:
miss_df = find_missing_percent(train)
'''Displays columns with missing values'''
display(miss_df[miss_df['PercentMissing']>0.0])
print("\n")
print(f"Number of columns with missing values:{str(miss_df[miss_df['PercentMissing']>0.0].shape[0])}")

,ColumnName,TotalMissingVals,PercentMissing
3,LotFrontage,259,17.74
6,Alley,1369,93.77
25,MasVnrType,872,59.73
26,MasVnrArea,8,0.55
30,BsmtQual,37,2.53
31,BsmtCond,37,2.53
32,BsmtExposure,38,2.60
33,BsmtFinType1,37,2.53
35,BsmtFinType2,38,2.60
42,Electrical,1,0.07




Number of columns with missing values:19


In [6]:
drop_cols = miss_df[miss_df['PercentMissing'] >40.0].ColumnName.tolist()
print(f"Number of columns with more than 40%: {len(drop_cols)}")
train = train.drop(drop_cols,axis=1)
test = test.drop(drop_cols,axis =1)

miss_df = miss_df[miss_df['ColumnName'].isin(train.columns)]
'''Columns to Impute'''
impute_cols = miss_df[miss_df['TotalMissingVals']>0.0].ColumnName.tolist()
res_col = miss_df[miss_df['TotalMissingVals']>0.0]
res_col

Number of columns with more than 40%: 6


,ColumnName,TotalMissingVals,PercentMissing
3,LotFrontage,259,17.74
26,MasVnrArea,8,0.55
30,BsmtQual,37,2.53
31,BsmtCond,37,2.53
32,BsmtExposure,38,2.60
33,BsmtFinType1,37,2.53
35,BsmtFinType2,38,2.60
42,Electrical,1,0.07
58,GarageType,81,5.55
59,GarageYrBlt,81,5.55


In [7]:
train = pd.concat([train, test])
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000.0


In [8]:
mylist_air = train['CentralAir'].tolist()

mylist_garage = train['GarageFinish'].tolist()

mylist_pave = train['PavedDrive'].tolist()

train_air = [1 if x == 'Y' else 0 for x in mylist_air]

train_garage = [1 if x == 'Fin' else 0 for x in mylist_garage]

train_pave = [1 if x == 'Y' else 0 for x in mylist_pave]

train['CentralAir'] = np.asarray(train_air)

train['GarageFinish'] = np.asarray(train_garage)

train['PavedDrive'] = np.asarray(train_pave)

In [9]:
numerical_train = [x for x in train.columns if train[x].dtype != 'object']

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [11]:
lr = LogisticRegression()

In [12]:
X = train[train['GarageFinish'] == 0][numerical_train].fillna(0)
y = train[train['GarageFinish'] == 0]['PavedDrive']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [13]:
lr.fit(X_train, y_train)

LogisticRegression()

In [14]:
roc_auc_score(y_test, lr.predict_proba(X_test)[:,1])

np.float64(0.9249613771794306)

In [18]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [16]:
knn_regressor = KNeighborsRegressor(n_neighbors=5)
knn_regressor.fit(X_train, y_train)

KNeighborsRegressor()

In [17]:
y_pred = knn_regressor.predict(X_test)

In [19]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [20]:
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Squared Error: 0.07945454545454546
R-squared: 0.15126903553299487


In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [22]:
model = LinearRegression()

In [24]:
kf = KFold(n_splits=5)
mse = cross_val_score(model, X, y, cv=kf, scoring='neg_mean_squared_error')

mse

array([-2.13825203e-28, -2.88476245e-30, -3.16257190e-30, -1.95440163e-30,
       -3.37182516e-30])